In [5]:
import random

import requests

TIMEOUT = 15

In [6]:
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/33.0.1750.517 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36",
]

In [10]:
url = "https://maratp.ratp.fr/newsletter/"
headers = {
        "User-Agent": random.choice(USER_AGENTS),
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "Accept-Language": "fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7",
        "Connection": "close",
    }
response = requests.get(url, headers=headers, timeout=TIMEOUT)
response.raise_for_status()
print(response.status_code, response.text)

200 
<!doctype html>
<html lang="fr-FR">
<head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <link rel="apple-touch-icon" sizes="180x180" href="https://maratp.ratp.fr/app/themes/maratp/public/apple-touch-icon.png">
    <link rel="icon" type="image/png" sizes="32x32" href="https://maratp.ratp.fr/app/themes/maratp/public/favicon-32x32.png">
    <link rel="icon" type="image/png" sizes="16x16" href="https://maratp.ratp.fr/app/themes/maratp/public/favicon-16x16.png">
    <link rel="manifest" href="https://maratp.ratp.fr/app/themes/maratp/public/site.webmanifest">
    <link rel="mask-icon" href="https://maratp.ratp.fr/app/themes/maratp/public/safari-pinned-tab.svg" color="#c929f9">
    <meta name="msapplication-TileColor" content="#ffffff">
    <meta name="theme-color" content="#c738f6">

            <!-- Tag commander vars -->
        <script>
            var tCPrivacyTagManager = "gtm";
        </script>

        <!-- Tag commander